In [32]:
import re
import string
import nltk

from nltk import FreqDist
from typing import Tuple, Callable
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [3]:
tweet_tokens[0]

['#FollowFriday',
 '@France_Inte',
 '@PKuchly57',
 '@Milipol_Paris',
 'for',
 'being',
 'top',
 'engaged',
 'members',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':)']

### Normalizando os dados

In [4]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

In [18]:
def lemmatize_sentence(normalization_func: Callable, tokens, stop_words: Tuple = ()):
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        word_ = normalization_func(word, tag)
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
    
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(word_, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            lemmatized_sentence.append(token.lower())
            
    return lemmatized_sentence


def remove_noise(token: str, tag: str):
    token_ = re.sub(
            r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|"\"(?:%[0-9a-fA-F][0-9a-fA-F]))+',
            '',
            token
    )
    token_ = re.sub(r'(@[A-Za-z0-9_]+)', '', token_)
    
    return token_

In [19]:
stop_words = stopwords.words('english')
print(lemmatize_sentence(remove_noise, tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [22]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(
        lemmatize_sentence(
            remove_noise,
            tokens,
            stop_words
        )
    )

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(
        lemmatize_sentence(
            remove_noise,
            tokens,
            stop_words
        )
    )

In [25]:
def get_all_words(cleaned_token_list):
    for tokens in cleaned_token_list:
        for token in tokens:
            yield token

In [33]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [35]:
all_neg_words = get_all_words(negative_cleaned_tokens_list)

freq_dist_neg = FreqDist(all_neg_words)
print(freq_dist_neg.most_common(10))

[(':(', 4585), (':-(', 501), ("i'm", 343), ('...', 332), ('get', 325), ('miss', 291), ('go', 275), ('please', 275), ('want', 246), ('like', 218)]
